In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, date

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('../')

from axia import shifted_beta_geometric
from axia import sbg_survival
from axia import util
from axia import report
from axia import loader

from importlib import reload

# Data: Load, Preprocess, and Split

In [2]:
jobber_data = loader.JobberDataLoader(path="../data/")
raw_data = jobber_data.get_data()
raw_data = raw_data.sample(100, random_state=189723)

In [3]:
%%time
reload(util)
data = util.SubscriptionData(
    data=raw_data,
    start_date_col="first_paying_date",
    end_date_col="last_churn_date",
    subscription_initial="initial_subscription_amt",
    subscription_current="current_subscription_amt",
    additional_cols=[raw_data["frequency"].astype("category")] + [
        raw_data[col].apply(func).astype("category")
        for col, func in jobber_data.transformations.items()
    ],
    split_at=datetime(2018, 1, 1),
#     split_at=datetime.now(),
)

CPU times: user 1.3 s, sys: 7 ms, total: 1.31 s
Wall time: 1.31 s


In [4]:
data.dtr.head()

,start_date,end_date,age,alive,subscription_initial,subscription_last,subscription_current,frequency,country,industry
account_id,,,,,,,,,,
14934,2017-04-01,NaT,9,1,96.0,96.0,96.0,monthly,united_states,other_industry
6233,2014-02-01,NaT,47,1,44.0,53.0,53.0,monthly,united_states,pressure_washing_service
44628,2016-01-01,2016-04-01,3,0,59.0,59.0,0.0,monthly,united_states,other
42851,2015-12-01,2017-12-01,24,0,48.0,48.0,0.0,annual,united_states,other
86400,2017-07-01,NaT,6,1,58.0,58.0,58.0,annual,united_states,residential_cleaning


In [5]:
data.dva.head()

,start_date,end_date,age,alive,subscription_initial,subscription_last,subscription_current,frequency,country,industry
account_id,,,,,,,,,,
113847,2018-07-01,NaT,1,1,145.0,145.0,145.0,monthly,canada,hvac
14934,2017-04-01,NaT,9,1,96.0,96.0,96.0,monthly,united_states,other_industry
115570,2018-07-01,NaT,1,1,291.0,145.0,145.0,monthly,united_states,construction_&_contracting
6233,2014-02-01,NaT,47,1,44.0,53.0,53.0,monthly,united_states,pressure_washing_service
134511,2019-02-01,NaT,1,1,78.0,78.0,78.0,monthly,united_states,hvac


In [6]:
dtr = data.dtr.join(data.features()).dropna(subset=['month_one_retention_trend_s3', 'month_one_retention_trend_s12'])
dva = data.dva.join(data.features())

In [34]:
reload(sbg_survival)

features = [
    "country",
    "industry",
    "frequency",
    "month_one_retention_trend_s3",
    "month_one_retention_trend_s12",
]

sbg = sbg_survival.SBGSurvival(
    age='age',
    alive='alive',
    features=features,
    gamma=1e-3,
    verbose=False
)
sbg.fit(df=dtr)

predicted_retention = sbg.predict_survival(dva, n_periods=3)
predicted_retention

probability
account_id age period             
956        65  0               NaN
           66  1               NaN
           67  2               NaN
1802       38  0          0.000000
           39  1          0.000000
           40  2          0.000000
4820       3   0          1.000000
           4   1          0.916805
           5   2          0.840702
6233       47  0          1.000000
           48  1          1.000000
           49  2          0.999999
7908       49  0          0.000000
           50  1          0.000000
           51  2          0.000000
8562       48  0          1.000000
           49  1          0.995650
           50  2          0.991406
8852       48  0          1.000000
           49  1          1.000000
           50  2          1.000000
10272      16  0          0.000000
           17  1          0.000000
           18  2          0.000000
12160      45  0          1.000000
           46  1          0.999843
           47  2          0.999690
14082      44  0          1.000000
           45  1          0.992364
           46  2          0.984933
...                            ...
124770     1   0          1.000000
           2   1          0.901720
           3   2          0.813112
126141     1   0          1.000000
           2   1          0.889392
           3   2          0.791506
131289     1   0          1.000000
           2   1          0.999990
           3   2          0.999981
131345     1   0          1.000000
           2   1          1.000000
           3   2          1.000000
131389     1   0          1.000000
           2   1          1.000000
           3   2          1.000000
131814     1   0          1.000000
           2   1          0.969407
           3   2          0.939781
133381     1   0          1.000000
           2   1          0.902638
           3   2          0.815045
133477     1   0          1.000000
           2   1          0.901720
           3   2          0.813112
133707     1   0          1.000000
           2   1          0.989042
           3   2          0.978204
134511     1   0          1.000000
           2   1          0.911380
           3   2          0.830617

[300 rows x 1 columns]